In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
data = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/ML homework/Copy of default of credit card clients.csv")

In [ ]:
data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


# Helper Function

In [ ]:
def preprocessing_(data):
    # get rid of missing values in MARRIAGE column
    data = data[data["MARRIAGE"] != 0]   
    
    # get rid of missing values in EDUCATION column
    data = data[data["EDUCATION"] != 0]    
    
    # discritize the AGE attribute to 6 interval
    data["AGE"] = pd.cut(data["AGE"], 6, labels=["AGE20", "AGE30", "AGE40", "AGE50", "AGE60", "AGE70"] )    
    #ordinal encoding
    encoder = OrdinalEncoder()
    # transform data
    data["AGE"] = encoder.fit_transform(np.array(data["AGE"]).reshape(-1, 1))
    
    return data

def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

        
def train_evaluate_val(model, x_train, y_train, folds, metrics):
    for metric in metrics:
        scores = cross_val_score(model, x_train, y_train, cv=folds, scoring = metric)
        print(f'{metric}:')
        print(f'Cross-validation: {scores}')
        print(f"Average: {scores.mean()}")
        print("----------------------------------------------------------")
    
def test_evaluate(model, x_test, y_test):
    prediction = model.predict(x_test)
    # prediction_prob = model.predict_proba(x_test)
    confusion = confusion_matrix(y_test, prediction)
    print("Confusion matrix",print_cm(confusion, ['Not pay', 'pay']))
    print('Accuracy: {:.2f}'.format(accuracy_score(y_test, prediction)))
    print('Precision: {:.2f}'.format(precision_score(y_test, prediction)))
    print('Recall: {:.2f}'.format(recall_score(y_test, prediction)))
    print('F1: {:.2f}'.format(f1_score(y_test, prediction)))
    print('AUC: {:.2f}'.format(roc_auc_score(y_test, prediction)))
    print(classification_report(y_test, prediction, target_names=['not pay', 'pay']))
    # precision_recall_curve(y_test, prediction_prob)

def optimize_model(model, x_train, y_train, parameters, metrics):
    for metric in metrics:
        print(f'{metric}:')
        grid_clf = GridSearchCV(model, param_grid = parameters, scoring = metric)
        grid_clf.fit(x_train, y_train)
        best_parameter = grid_clf.best_params_
        best_score = grid_clf.best_score_
        print(f"The best parameters: {best_parameter}\nThe best score: {best_score}\n")

# Prepare the data:

In [ ]:
data = preprocessing_(data)

In [ ]:
data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,0.0,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,0.0,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,1.0,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,1.0,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,3.0,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,1.0,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,2.0,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,1.0,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,2.0,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


# combinning features and generate a high level features:

Combination one: combin all the preivous payements history of every person which will make a new feature represent the dealy of every person for pay his bills 

Make the new feature:("payment_history")

In [ ]:
data["payment_delay"] = data['PAY_0'] + data['PAY_2'] + data['PAY_3'] + data['PAY_4'] + data['PAY_5'] + data['PAY_6'] 

In [ ]:
data["payment_delay"]

0       -2
1        3
2        0
3        0
4       -2
        ..
29995    0
29996   -4
29997    8
29998   -1
29999    0
Name: payment_delay, Length: 29932, dtype: int64

Make the new feature:("payment_delay_amount")

In [ ]:
data['payment_delay_amount'] = (data['BILL_AMT1'] - data['PAY_AMT1']) + (data['BILL_AMT2'] - data['PAY_AMT2']) + (data['BILL_AMT3'] - data['PAY_AMT3']) + (data['BILL_AMT4'] - data['PAY_AMT4']) + (data['BILL_AMT5'] - data['PAY_AMT5']) + (data['BILL_AMT6'] - data['PAY_AMT6'])

In [ ]:
data['payment_delay_amount']

0          7015
1         12077
2         90635
3        222946
4         50290
          ...  
29995    682799
29996      6692
29997     39196
29998    119430
29999    222566
Name: payment_delay_amount, Length: 29932, dtype: int64

Make the new feature:("total_payment_amount")

In [ ]:
data["total_payment_amount"] = data['PAY_AMT1'] + data['PAY_AMT2'] + data['PAY_AMT3'] + data['PAY_AMT4'] + data['PAY_AMT5'] + data['PAY_AMT6'] 

In [ ]:
data["total_payment_amount"]

0           689
1          5000
2         11018
3          8388
4         59049
          ...  
29995     42550
29996     14490
29997     31300
29998    147181
29999      8308
Name: total_payment_amount, Length: 29932, dtype: int64

# Split the data:

In [ ]:
y = undersampling_data["default payment next month"]
data = undersampling_data.drop("default payment next month", axis=1)

# x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(data[['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'payment_delay', 'payment_delay_amount', 'total_payment_amount']], y, test_size=0.2, random_state=42)#'LIMIT_BAL' 'delay_amount'

Normalization:

In [ ]:
scaler = preprocessing.StandardScaler().fit(np.array(x_train["payment_delay_amount"]).reshape(-1, 1))
x_train["payment_delay_amount"] = scaler.transform(np.array(x_train["payment_delay_amount"]).reshape(-1, 1))
x_test["payment_delay_amount"] = scaler.transform(np.array(x_test["payment_delay_amount"]).reshape(-1, 1))

scaler1 = preprocessing.StandardScaler().fit(np.array(x_train["total_payment_amount"]).reshape(-1, 1))
x_train["total_payment_amount"] = scaler1.transform(np.array(x_train["total_payment_amount"]).reshape(-1, 1))
x_test["total_payment_amount"] = scaler1.transform(np.array(x_test["total_payment_amount"]).reshape(-1, 1))

scaler2 = preprocessing.StandardScaler().fit(np.array(x_train["LIMIT_BAL"]).reshape(-1, 1))
x_train["LIMIT_BAL"] = scaler2.transform(np.array(x_train["LIMIT_BAL"]).reshape(-1, 1))
x_test["LIMIT_BAL"] = scaler2.transform(np.array(x_test["LIMIT_BAL"]).reshape(-1, 1))

x_train

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,payment_delay,payment_delay_amount,total_payment_amount
1481,1.109952,1,2,1,1.0,0,2.877147,0.561698
565,-1.058180,2,2,1,0.0,0,0.074317,-0.221095
11589,-0.094566,2,1,2,1.0,-6,-0.653865,0.243542
5378,1.993265,2,1,1,1.0,-12,-0.688092,-0.080351
8287,-1.138482,2,3,1,2.0,-2,-0.672366,-0.513118
...,...,...,...,...,...,...,...,...
1971,-0.496072,1,2,2,0.0,0,0.332965,-0.268963
9886,-1.058180,1,2,2,0.0,0,-0.431132,-0.443413
285,-0.977879,1,2,2,0.0,0,-0.223417,-0.395316
4923,0.547844,2,3,1,2.0,0,1.760967,0.416281


# undersampling:

In [ ]:
# the unbalanecd classes
class_0 = x_train[y_train == 0]
class_1 = x_train[y_train == 1]

print("before undersampling:\n")
print(f"not payed class: {len(class_0)}\n") #18641

print(f"payed class: {len(class_1)}\n") #5304

# under sample the dominent class(not payed)
class_0 = class_0[0:6001]
print("after undersampling:\n")
print(f"not payed class: {len(class_0)}\n")
# adding label as not payed
y_0 = np.zeros(len(class_0))
y_0 = pd.DataFrame(y_0, columns=["label"])

print(f"payed class: {len(class_1)}\n")
# adding label as payed
y_1 = np.ones(len(class_1))
y_1 = pd.DataFrame(y_1, columns=["label"])

# concatinate the new balanced data
undersampling_data_samples = pd.concat([class_0, class_1], ignore_index=True)
undersampling_data_label = pd.concat([y_0, y_1], ignore_index=True)
undersampling_data_samples["label"] = undersampling_data_label


# shuffling the data
undersampling_data = shuffle(undersampling_data_samples)
undersampling_data

before undersampling:

not payed class: 23301

payed class: 6631

after undersampling:

not payed class: 7000

payed class: 6631



In [ ]:
y_train = undersampling_data["label"]
x_train = undersampling_data.drop("label", axis=1)

# Train & Evaluate: 

In [ ]:
metrics = ["accuracy", 'precision', 'recall', 'f1', 'roc_auc']

Logistic Regression:

In [ ]:
l_g_grid_values = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
l_g = LogisticRegression()

train and evaluate using cross validation:

In [ ]:
train_evaluate_val(l_g, x_train, y_train, 5, metrics)

accuracy:
Cross-validation: [0.64144888 0.64465841 0.64007336 0.63594681 0.65825688]
Average: 0.6440768690399573
----------------------------------------------------------
precision:
Cross-validation: [0.64632885 0.65167364 0.64893617 0.64308342 0.6580773 ]
Average: 0.6496198776184503
----------------------------------------------------------
recall:
Cross-validation: [0.58685446 0.58497653 0.57276995 0.57183099 0.62406015]
Average: 0.5880984150517138
----------------------------------------------------------
f1:
Cross-validation: [0.61515748 0.61652647 0.6084788  0.60536779 0.64061746]
Average: 0.6172296022412295
----------------------------------------------------------
roc_auc:
Cross-validation: [0.68322101 0.68906726 0.68550112 0.67630202 0.7008078 ]
Average: 0.6869798410661415
----------------------------------------------------------


optimize the model using grid search: 

In [ ]:
l_g = LogisticRegression()
optimize_model(l_g, x_train, y_train, l_g_grid_values, metrics)

accuracy:
The best parameters: {'C': 0.001}
The best score: 0.6503124145560701

precision:
The best parameters: {'C': 0.001}
The best score: 0.6615117013085914

recall:
The best parameters: {'C': 10}
The best score: 0.5882862084789438

f1:
The best parameters: {'C': 0.001}
The best score: 0.6187621965257659

roc_auc:
The best parameters: {'C': 100}
The best score: 0.6869874122746921



Final evaluation on the test data:

In [ ]:
l_g = LogisticRegression(C=10)
l_g.fit(x_train, y_train)
test_evaluate(l_g, x_test, y_test)

            Not pay     pay 
    Not pay   988.0   432.0 
        pay   523.0   784.0 
Confusion matrix None
Accuracy: 0.65
Precision: 0.64
Recall: 0.60
F1: 0.62
AUC: 0.65
              precision    recall  f1-score   support

     not pay       0.65      0.70      0.67      1420
         pay       0.64      0.60      0.62      1307

    accuracy                           0.65      2727
   macro avg       0.65      0.65      0.65      2727
weighted avg       0.65      0.65      0.65      2727



Decision Tree:

In [ ]:
d_t = DecisionTreeClassifier()

In [ ]:
d_t_grid_values = {'max_depth': [10,20,30,50,100],
                   'criterion': ["gini", "entropy"],
                   'splitter': ["best", "random"],}

train and evaluate using cross validation:

In [ ]:
train_evaluate_val(d_t, x_train, y_train, 5, metrics)

accuracy:
Cross-validation: [0.60614397 0.62769372 0.62219166 0.60981201 0.61376147]
Average: 0.6159205650131032
----------------------------------------------------------
precision:
Cross-validation: [0.59828244 0.62183236 0.61486486 0.59337626 0.59925094]
Average: 0.6055213735132822
----------------------------------------------------------
recall:
Cross-validation: [0.58779343 0.61502347 0.5971831  0.61784038 0.59962406]
Average: 0.603492887147446
----------------------------------------------------------
f1:
Cross-validation: [0.60412758 0.60669856 0.61414333 0.60893855 0.59388235]
Average: 0.6055580753018306
----------------------------------------------------------
roc_auc:
Cross-validation: [0.61182838 0.63341495 0.61995768 0.60206766 0.60594952]
Average: 0.6146436366738443
----------------------------------------------------------


optimize the model using grid search: 

In [ ]:
d_t = DecisionTreeClassifier()
                
optimize_model(d_t, x_train, y_train, d_t_grid_values, metrics)

accuracy:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'splitter': 'best'}
The best score: 0.6778264746833578

precision:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'splitter': 'random'}
The best score: 0.6945607389187741

recall:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'splitter': 'best'}
The best score: 0.6470666101874404

f1:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'splitter': 'best'}
The best score: 0.6604815318890209

roc_auc:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'splitter': 'random'}
The best score: 0.7206349903799328



Final evaluation on the test data:

In [ ]:
d_t = DecisionTreeClassifier(max_depth=10, criterion = 'entropy', splitter = 'best')
d_t.fit(x_train, y_train)
test_evaluate(d_t, x_test, y_test)

            Not pay     pay 
    Not pay  1042.0   378.0 
        pay   482.0   825.0 
Confusion matrix None
Accuracy: 0.68
Precision: 0.69
Recall: 0.63
F1: 0.66
AUC: 0.68
              precision    recall  f1-score   support

     not pay       0.68      0.73      0.71      1420
         pay       0.69      0.63      0.66      1307

    accuracy                           0.68      2727
   macro avg       0.68      0.68      0.68      2727
weighted avg       0.68      0.68      0.68      2727



Random Forest:

In [ ]:
r_f = RandomForestClassifier()

In [ ]:
r_f_grid_values = {'max_depth': [10, 20, 30, 50, 100],
                    'n_estimators': [10, 20, 40, 80, 100],
                    'criterion': ["gini", "entropy"]}

train and evaluate using cross validation:

In [ ]:
train_evaluate_val(r_f, x_train, y_train, 5, metrics)

accuracy:
Cross-validation: [0.68500688 0.68133884 0.69188446 0.68042182 0.69220183]
Average: 0.6861707658720644
----------------------------------------------------------
precision:
Cross-validation: [0.6884273  0.68237705 0.69366563 0.67889908 0.69330669]
Average: 0.6873351506008321
----------------------------------------------------------
recall:
Cross-validation: [0.65821596 0.63098592 0.61971831 0.63474178 0.66541353]
Average: 0.6418151011331144
----------------------------------------------------------
f1:
Cross-validation: [0.67436267 0.65979381 0.65340627 0.64702995 0.68507752]
Average: 0.663934043942812
----------------------------------------------------------
roc_auc:
Cross-validation: [0.74235238 0.73954011 0.74265107 0.73792216 0.75329874]
Average: 0.7431528903345243
----------------------------------------------------------


optimize the model using grid search: 

In [ ]:
r_f = RandomForestClassifier()
                
optimize_model(r_f, x_train, y_train, r_f_grid_values, metrics)

accuracy:
The best parameters: {'criterion': 'gini', 'max_depth': 10, 'n_estimators': 100}
The best score: 0.7050631601529472

precision:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 100}
The best score: 0.7162975445055613

recall:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 80}
The best score: 0.6557126972360479

f1:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 80}
The best score: 0.6846428560370592

roc_auc:
The best parameters: {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 100}
The best score: 0.7648397501591325



Final evaluation on the test data:

In [ ]:
r_f = RandomForestClassifier(max_depth = 10, n_estimators = 80, criterion = 'entropy')
r_f.fit(x_train, y_train)
test_evaluate(r_f, x_test, y_test)

            Not pay     pay 
    Not pay  1060.0   360.0 
        pay   450.0   857.0 
Confusion matrix None
Accuracy: 0.70
Precision: 0.70
Recall: 0.66
F1: 0.68
AUC: 0.70
              precision    recall  f1-score   support

     not pay       0.70      0.75      0.72      1420
         pay       0.70      0.66      0.68      1307

    accuracy                           0.70      2727
   macro avg       0.70      0.70      0.70      2727
weighted avg       0.70      0.70      0.70      2727



# Deep Neural network:

Metrics:

In [ ]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', precision_m, recall_m, f1_m,])
# fit the keras model on the dataset
model.fit(np.array(x_train), np.array(y_train), epochs=100, batch_size=100, validation_split=0.2)
model.evaluate(x=x_test, y=y_test)


Epoch 1/100
873/873 [==============================] - 3s 2ms/step - loss: 0.6383 - accuracy: 0.6254 - precision_m: 0.6505 - recall_m: 0.6353 - f1_m: 0.5893 - val_loss: 0.5882 - val_accuracy: 0.6919 - val_precision_m: 0.7438 - val_recall_m: 0.5299 - val_f1_m: 0.5905
Epoch 2/100
873/873 [==============================] - 2s 2ms/step - loss: 0.5977 - accuracy: 0.6816 - precision_m: 0.7419 - recall_m: 0.5138 - f1_m: 0.5749 - val_loss: 0.5843 - val_accuracy: 0.6942 - val_precision_m: 0.7463 - val_recall_m: 0.5379 - val_f1_m: 0.5961
Epoch 3/100
873/873 [==============================] - 2s 2ms/step - loss: 0.5989 - accuracy: 0.6866 - precision_m: 0.7213 - recall_m: 0.5397 - f1_m: 0.5853 - val_loss: 0.5807 - val_accuracy: 0.6992 - val_precision_m: 0.6878 - val_recall_m: 0.6567 - val_f1_m: 0.6483
Epoch 4/100
873/873 [==============================] - 1s 2ms/step - loss: 0.5900 - accuracy: 0.6906 - precision_m: 0.7081 - recall_m: 0.5778 - f1_m: 0.6075 - val_loss: 0.5877 - val_accuracy: 0.6809 

[0.6001665592193604,
 0.6839017271995544,
 0.6773856282234192,
 0.6613942980766296,
 0.6617529988288879]